In [3]:
# Importando las librerías
using DataFrames
using CSV

In [4]:
# Leyendo los datos de entrada
df = CSV.read("Datos_Aparejo_W-31.csv", DataFrame);

In [9]:
# Mostrando los primeros 5 renglones de la tabla
last(df)

Row,#,D_Ext,Dint_,Long,Cima,Base,Lib,Rt
,Int64,Float64,String7,Float64,Float64,Float64,Float64,Int64
23,23,4.42,-,2.08,3715.0,3717.08,9.2,246000


In [7]:
# Datos
d_fc = 0.86         # Densidad del fluido de control- diesel 
FF = 1-(d_fc/7.85)  # Factor de flotación
FS_p = 80           # Factor de seguridad - (porcentaje)
FS = 1/(FS_p/100);   # Factor de seguridad - adim

$ F_{f} = 1 - \frac{\rho_{fc}}{7.85} $ 

$ R_{t80} = \frac{R_{t}}{FS} $

$ W_{aire} = \frac{Long \cdot Lib}{0.3048} $

$ W_{flot} = W_{aire} \cdot FF $

$ MOP_{aire} = R_{t80} - W_{aire\_acum} $

$ MOP_{flot} = R_{t80} - W_{flot\_acum} $

$ MOP_{aire\_ton} = \frac {0.4536 \cdot MOP_{aire}}{1000} $

$ MOP_{flot\_ton} = \frac {0.4536 \cdot MOP_{flot}}{1000} $

In [19]:
# Datos de entrada
Rt = 246_000      # (lbs)
long = 2.08       # (m)
Lib = 9.2         # (lb/pie)
W_ant_aire = 0    # (lbs)
W_ant_flot = 0    # (lbs)

# Resistencia a la tensión al 80% (lbs)
Rt_80 = 0.8*Rt

# Pesos en el aire y flotado (lbs)
W_aire = long*Lib/0.3048
W_flot = W_aire*FF

# Pesos acumulados en el aire y flotado (lbs)
W_aire_acum = W_ant_aire + W_aire
W_flot_acum = W_ant_flot + W_flot

# Margen de jalón (ton)
MOP_aire = (Rt_80 - W_aire_acum)*0.4536*1e-3
MOP_flot = (Rt_80 - W_flot_acum)*0.4536*1e-3

println("El margen de jalón en el aire al 80% es: $MOP_aire ton")
println("El margen de jalón flotado al 80% es: $MOP_flot ton")

El margen de jalón en el aire al 80% es: 89.24000201574803 ton
El margen de jalón flotado al 80% es: 89.24312189682533 ton


In [21]:
# Realizando los cálculos:
# Resistencia a la tensión con Factor de Seguridad - lbs
df[!, :Rt_80] = df.Rt/FS

# Pesos en el aire y flotado - lbs
df[!, :W_aire] = (df.Long) .* (df.Lib)/0.3048
df[!, :W_flot] = df.W_aire*FF

# Pesos acumulados - lbs
W_aire_acum, W_flot_acum = Any[], Any[]

for fila in 1:23
    push!(W_aire_acum, sum(df.W_aire[fila:end]))
    push!(W_flot_acum, sum(df.W_flot[fila:end]))
end

df[!, :W_aire_acum] = Float64.(W_aire_acum)
df[!, :W_flot_acum] = Float64.(W_flot_acum)

# Margen de Jalón - lbs
df[!, :MOP_aire] = df.Rt_80 - df.W_aire_acum
df[!, :MOP_flot] = df.Rt_80 - df.W_flot_acum

# Margen de Jalón - ton
df[!, :MOP_aire_ton] = df.MOP_aire*0.4536*1e-3
df[!, :MOP_flot_ton] = df.MOP_flot*0.4536*1e-3

last(df, 1)

Row,#,D_Ext,Dint_,Long,Cima,Base,Lib,Rt,Rt_80,W_aire,W_flot,W_aire_acum,W_flot_acum,MOP_aire,MOP_flot,MOP_aire_ton,MOP_flot_ton
,Int64,Float64,String7,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,23,4.42,-,2.08,3715.0,3717.08,9.2,246000,196800.0,62.7822,55.9041,62.7822,55.9041,1.96737e5,1.96744e5,89.24,89.2431


In [22]:
first(sort(df, [:MOP_flot_ton]), 1)

Row,#,D_Ext,Dint_,Long,Cima,Base,Lib,Rt,Rt_80,W_aire,W_flot,W_aire_acum,W_flot_acum,MOP_aire,MOP_flot,MOP_aire_ton,MOP_flot_ton
,Int64,Float64,String7,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,7,4.5,2.992,0.39,225.53,225.92,9.2,246000,196800.0,11.7717,10.482,1.1041e5,98314.1,86390.1,98485.9,39.1865,44.6732
